# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from config import *

In [3]:
features = ['Was_home',
 'Rating_difference',
 'Value',
 'Transfers_balance',
 'Avg_FPL_points',
 'Min_2',
 'Gls_2',
 'Ast_2',
 'xG_2',
 'xA_2',
 'Team_CS_2',
 'Team_score_2',
 'Opp_score_2',
 'Team_result_2',
 'FPL_points_2',
 'Min_4',
 'Gls_4',
 'Ast_4',
 'xG_4',
 'xA_4',
 'Team_CS_4',
 'Team_score_4',
 'Opp_score_4',
 'Team_result_4',
 'FPL_points_4',
 'Min_30',
 'Gls_30',
 'Ast_30',
 'xG_30',
 'xA_30',
 'Team_CS_30',
 'Team_score_30',
 'Opp_score_30',
 'Team_result_30',
 'FPL_points_30',
 'DEF',
 'FWD',
 'MID']

info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]

In [4]:
len(features)

38

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (50) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
X_test = df[df["Season"] == CURRENT_SEASON][features]

In [7]:
df.columns

Index(['Unnamed: 0', 'Ast_2', 'Ast_30', 'Ast_4', 'Avg_FPL_points', 'DEF',
       'FPL_points', 'FPL_points_2', 'FPL_points_30', 'FPL_points_4',
       'FPL_pos', 'FWD', 'GW', 'Gls_2', 'Gls_30', 'Gls_4', 'MID', 'Min_2',
       'Min_30', 'Min_4', 'Name_original', 'Opp_rating', 'Opp_score_2',
       'Opp_score_30', 'Opp_score_4', 'Opponent', 'Rating_difference',
       'Season', 'Team', 'Team_CS_2', 'Team_CS_30', 'Team_CS_4', 'Team_rating',
       'Team_result_2', 'Team_result_30', 'Team_result_4', 'Team_score_2',
       'Team_score_30', 'Team_score_4', 'Transfers_balance', 'Value',
       'Was_home', 'xA_2', 'xA_30', 'xA_4', 'xG_2', 'xG_30', 'xG_4', 'xP',
       'Finished', 'Date', 'Price'],
      dtype='object')

In [8]:
X_test.columns

Index(['Was_home', 'Rating_difference', 'Value', 'Transfers_balance',
       'Avg_FPL_points', 'Min_2', 'Gls_2', 'Ast_2', 'xG_2', 'xA_2',
       'Team_CS_2', 'Team_score_2', 'Opp_score_2', 'Team_result_2',
       'FPL_points_2', 'Min_4', 'Gls_4', 'Ast_4', 'xG_4', 'xA_4', 'Team_CS_4',
       'Team_score_4', 'Opp_score_4', 'Team_result_4', 'FPL_points_4',
       'Min_30', 'Gls_30', 'Ast_30', 'xG_30', 'xA_30', 'Team_CS_30',
       'Team_score_30', 'Opp_score_30', 'Team_result_30', 'FPL_points_30',
       'DEF', 'FWD', 'MID'],
      dtype='object')

In [24]:
# df[df["FPL_pos"] == "GK"]

In [10]:
def new_predict(X):
    XGB_COMPONENT = 0.45
    OVERALL_COMPONENT = 0.5
    FORM_COMPONENT = 0.25
    
    model_pred = np.array(model.predict2(X))
    overall_pred = np.array([row["Avg_FPL_points"] for i, row in X.iterrows()])
    form_pred = np.array([row["FPL_points_4"] for i, row in X.iterrows()])
    
    return np.add( model_pred * XGB_COMPONENT, overall_pred * OVERALL_COMPONENT, form_pred * FORM_COMPONENT )

file = open("models/GBR.pkl",'rb')
model = pickle.load(file)

In [11]:
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

# Predicting

In [12]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [13]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [14]:
preds = get_predictions(model, df, X_test)

15033 (15033, 52)


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [15]:
# preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == 2)].head(30)

In [16]:
# next gameweek
preds[preds["GW"] == 3].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
1973,Bukayo Saka,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,86.0,5.257550
10530,Mohamed Salah,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,MID,125.0,5.086303
4437,Erling Haaland,3.0,2023-24,Manchester City,Sheffield United,0.0,2080.850586,1637.473755,FWD,140.0,5.054771
5004,Gabriel Martinelli Silva,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,80.0,5.009655
2049,Callum Wilson,3.0,2023-24,Newcastle Utd,Liverpool,1.0,1887.064819,1943.976074,FWD,80.0,4.450448
1860,Bruno Borges Fernandes,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1877.037842,1671.868774,MID,85.0,4.401895
115,Aaron Ramsdale,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,GK,50.0,4.233747
837,Andrew Robertson,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,DEF,65.0,4.232021
3298,David Raya Martin,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,GK,50.0,4.160720
13330,Solly March,3.0,2023-24,Brighton,West Ham,1.0,1832.364258,1777.551270,MID,65.0,4.157702


In [17]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
10555,Mohamed Salah,28.0,2023-24,Liverpool,Manchester City,1.0,1943.976074,2080.850586,MID,125.0,5.972085
10529,Mohamed Salah,2.0,2023-24,Liverpool,Bournemouth,1.0,1943.976074,1662.536621,MID,125.0,5.909197
10541,Mohamed Salah,14.0,2023-24,Liverpool,Fulham,1.0,1943.976074,1745.365723,MID,125.0,5.909197
10545,Mohamed Salah,18.0,2023-24,Liverpool,Arsenal,1.0,1943.976074,1921.462769,MID,125.0,5.909197
10547,Mohamed Salah,20.0,2023-24,Liverpool,Newcastle Utd,1.0,1943.976074,1887.064819,MID,125.0,5.909197
10549,Mohamed Salah,22.0,2023-24,Liverpool,Chelsea,1.0,1943.976074,1791.018188,MID,125.0,5.909197
10551,Mohamed Salah,24.0,2023-24,Liverpool,Burnley,1.0,1943.976074,1722.802979,MID,125.0,5.909197
10553,Mohamed Salah,26.0,2023-24,Liverpool,Luton,1.0,1943.976074,1602.327637,MID,125.0,5.909197
10557,Mohamed Salah,30.0,2023-24,Liverpool,Brighton,1.0,1943.976074,1832.364258,MID,125.0,5.909197
10558,Mohamed Salah,31.0,2023-24,Liverpool,Sheffield United,1.0,1943.976074,1637.473755,MID,125.0,5.909197


In [18]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Mohamed Salah,209.116494
Erling Haaland,201.685501
Gabriel Martinelli Silva,182.218164
Bukayo Saka,179.151286
Andrew Robertson,168.253284
Miguel Almirón Rejala,168.086055
Trent Alexander-Arnold,167.146159
Callum Wilson,161.406825
Alisson Ramses Becker,160.825246


# Saving to file

In [19]:
preds.to_csv("predictions/preds_next_season.csv", index=False)

# Importance

In [20]:
# import shap
# explainer = shap.Explainer(model.predict, X_test[9000:10000])
# shap_values = explainer(X_test[9000:10000])

In [21]:
# shap.plots.bar(shap_values, max_display=15)

In [22]:
# # Salah vs Nottingham Forest
# shap.plots.waterfall(shap_values[502])

In [23]:
# shap.plots.waterfall(shap_values[470])